# Contents:

1. ANN1: Using everything - 70 features
2. ANN2: Without spectral features - 68 features
3. ANN3: Using both spectral and tempo features with feature selection

In [27]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data-v4.csv', index_col=0)

In [3]:
df.head(3)

,ZCR,Energy,Entropy of Energy,Spectral Centroid,Spectral Spread,Spectral Entropy,Spectral Flux,Spectral Rolloff,MFCC.1,MFCC.2,...,CV.7_SD,CV.8_SD,CV.9_SD,CV.10_SD,CV.11_SD,Chroma Deviation_SD,Tempo Feature1,Tempo Feature2,Valence,Arousal
Song ID,,,,,,,,,,,,,,,,,,,,,
0,0.234463,0.074886,0.792056,0.153942,0.077575,0.305852,0.140355,0.343998,0.591378,0.579080,...,0.189526,0.193013,0.273233,0.419740,0.482732,0.170282,0.142857,0.320790,-0.4,0.5
1,0.192285,0.096489,0.814660,0.136038,0.088316,0.241364,0.107631,0.282864,0.561162,0.630659,...,0.085899,0.163944,0.437033,0.428849,0.427423,0.146012,0.142857,0.271091,0.5,-0.5
2,0.529244,0.184599,0.562987,0.414212,0.137845,0.759576,0.184610,0.761827,0.481685,0.165364,...,0.251068,0.029417,0.195861,0.598474,0.164410,0.329922,0.142857,0.250898,-0.2,0.4


In [4]:
data_tr1 = df.iloc[:264, 0:70]
data_ts1 = df.iloc[264:, 0:70]
valence_tr1 = df.iloc[:264, 70]
arousal_tr1 = df.iloc[:264, 71]
valence_ts1 = df.iloc[264:, 70]
arousal_ts1 = df.iloc[264:, 71]

data_tr2 = df.iloc[:264, 0:68]
data_ts2 = df.iloc[264:, 0:68]
valence_tr2 = df.iloc[:264, 70]
arousal_tr2 = df.iloc[:264, 71]
arousal_ts2 = df.iloc[264:, 71]
valence_ts2 = df.iloc[264:, 70]

data3 = df.loc[:, ['Tempo Feature1',
 'Spectral Centroid',
 'ZCR',
 'Spectral Entropy',
 'CV.7',
 'Spectral Flux',
 'Spectral Rolloff',
 'MFCC.4',
 'MFCC.2',
 'CV.5',
 'Entropy of Energy',
 'MFCC.3',
 'CV.8',
 'Tempo Feature2',
 'MFCC.13',
 'Spectral Centroid_SD',
 'ZCR_SD',
 'Spectral Entropy_SD',
 'CV.7_SD',
 'Spectral Flux_SD',
 'Spectral Rolloff_SD',
 'MFCC.4_SD',
 'MFCC.2_SD',
 'CV.5_SD',
 'Entropy of Energy_SD',
 'MFCC.3_SD',
 'CV.8_SD',
 'MFCC.13_SD']]
data_tr3 = data3.iloc[:264, :]
data_ts3 = data3.iloc[264:, :]
valence_tr3 = df.iloc[:264, 70]
arousal_tr3 = df.iloc[:264, 71]
arousal_ts3 = df.iloc[264:, 71]
valence_ts3 = df.iloc[264:, 70]

In [5]:
from keras import models
from keras.layers import Activation, Dropout, Dense, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


# ANN1

In [6]:
def create_network():
    
    # instantiate model
    model = models.Sequential()

    # we can think of this chunk as the input layer
    model.add(Dense(90, input_dim=data_tr1.shape[1])) #change this bit for the different size of imput features
    model.add(Activation('relu'))


    # we can think of this chunk as the hidden layers    
    model.add(Dense(90, kernel_initializer = 'uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(90, kernel_initializer = 'uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    #Output Layer

    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['mae'])
    
    return model

In [7]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 nb_epoch=10, 
                                 batch_size=30, 
                                 verbose=0)
params = {'batch_size' : [30, 40, 50]}
grid_searc_nn1_valence = GridSearchCV(estimator = neural_network, param_grid = params, cv = 10, scoring = 'neg_mean_squared_error')
grid_searc_nn1_valence.fit(data_tr1, valence_tr1)
valence_pred_ts1 = grid_searc_nn1_valence.predict(data_ts1)
valence_pred_tr1 = grid_searc_nn1_valence.predict(data_tr1)

/home/broda/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [8]:
grid_searc_nn1_arousal = GridSearchCV(estimator = neural_network, param_grid = params, cv = 10, scoring = 'neg_mean_squared_error')
grid_searc_nn1_arousal.fit(data_tr1, arousal_tr1)
arousal_pred_ts1 = grid_searc_nn1_arousal.predict(data_ts1)
arousal_pred_tr1 = grid_searc_nn1_arousal.predict(data_tr1)

/home/broda/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# ANN2

In [11]:
def create_network():
    
    # instantiate model
    model = models.Sequential()

    # we can think of this chunk as the input layer
    model.add(Dense(90, input_dim=data_tr2.shape[1])) #change this bit for the different size of imput features
    model.add(Activation('relu'))


    # we can think of this chunk as the hidden layers    
    model.add(Dense(90, kernel_initializer = 'uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(90, kernel_initializer = 'uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    #Output Layer

    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['mae'])
    
    return model

neural_network = KerasRegressor(build_fn=create_network, 
                                 nb_epoch=10, 
                                 batch_size=30, 
                                 verbose=0)

params = {'batch_size' : [30, 40, 50]}
grid_searc_nn2_valence = GridSearchCV(estimator = neural_network, param_grid = params, cv = 10, scoring = 'neg_mean_squared_error')

In [12]:
grid_searc_nn2_valence.fit(data_tr2, valence_tr2)
valence_pred_ts2 = grid_searc_nn2_valence.predict(data_ts2)
valence_pred_tr2 = grid_searc_nn2_valence.predict(data_tr2)

/home/broda/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [13]:
grid_searc_nn2_arousal = GridSearchCV(estimator = neural_network, param_grid = params, cv = 10, scoring = 'neg_mean_squared_error')
grid_searc_nn2_arousal.fit(data_tr2, arousal_tr2)
arousal_pred_ts2 = grid_searc_nn2_arousal.predict(data_ts2)
arousal_pred_tr2 = grid_searc_nn2_arousal.predict(data_tr2)

/home/broda/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# ANN3

In [14]:
def create_network():
    
    # instantiate model
    model = models.Sequential()

    # we can think of this chunk as the input layer
    model.add(Dense(90, input_dim=data_tr3.shape[1])) #change this bit for the different size of imput features
    model.add(Activation('relu'))


    # we can think of this chunk as the hidden layers    
    model.add(Dense(90, kernel_initializer = 'uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(90, kernel_initializer = 'uniform'))
    #model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    #Output Layer

    model.add(Dense(1))

    model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['mae'])
    
    return model

neural_network = KerasRegressor(build_fn=create_network, 
                                 nb_epoch=10, 
                                 batch_size=30, 
                                 verbose=0)

params = {'batch_size' : [30, 40, 50]}
grid_searc_nn3_valence = GridSearchCV(estimator = neural_network, param_grid = params, cv = 10, scoring = 'neg_mean_squared_error')

In [15]:
grid_searc_nn3_valence.fit(data_tr3, valence_tr3)
valence_pred_ts3 = grid_searc_nn3_valence.predict(data_ts3)
valence_pred_tr3 = grid_searc_nn3_valence.predict(data_tr3)

/home/broda/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [17]:
grid_searc_nn3_arousal = GridSearchCV(estimator = neural_network, param_grid = params, cv = 10, scoring = 'neg_mean_squared_error')
grid_searc_nn3_arousal.fit(data_tr3, arousal_tr3)
arousal_pred_ts3 = grid_searc_nn3_arousal.predict(data_ts3)
arousal_pred_tr3 = grid_searc_nn3_arousal.predict(data_tr3)

/home/broda/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# Performance Eval

In [18]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

In [19]:
#Arousal Error(mae) vs dataset 1,2,3 
print('1. Training Error :' + str(mae(arousal_tr1,arousal_pred_tr1)) + ' Test Error :' + str(mae(arousal_ts1,arousal_pred_ts1)))
print('2. Training Error :' + str(mae(arousal_tr2,arousal_pred_tr2)) + ' Test Error :' + str(mae(arousal_ts2,arousal_pred_ts2)))
print('3. Training Error :' + str(mae(arousal_tr3,arousal_pred_tr3)) + ' Test Error :' + str(mae(arousal_ts3,arousal_pred_ts3)))

1. Training Error :0.431315770158262 Test Error :0.36780555232456247
2. Training Error :0.4264300572770563 Test Error :0.3556250941572768
3. Training Error :0.42885959667584045 Test Error :0.3864292765374889


In [20]:
#Arousal Error(mse) vs dataset 1,2,3 
print('1. Training Error :' + str(mse(arousal_tr1,arousal_pred_tr1)) + ' Test Error :' + str(mse(arousal_ts1,arousal_pred_ts1)))
print('2. Training Error :' + str(mse(arousal_tr2,arousal_pred_tr2)) + ' Test Error :' + str(mse(arousal_ts2,arousal_pred_ts2)))
print('3. Training Error :' + str(mse(arousal_tr3,arousal_pred_tr3)) + ' Test Error :' + str(mse(arousal_ts3,arousal_pred_ts3)))

1. Training Error :0.22191622397075572 Test Error :0.17540511677061996
2. Training Error :0.228955854177398 Test Error :0.17125678453047
3. Training Error :0.22331568497529164 Test Error :0.1981770139802427


In [21]:
#Arousal Error(r2) vs dataset 1,2,3 
print('1. Training r2 score :' + str(r2(arousal_tr1,arousal_pred_tr1)) + ' Test r2 score :' + str(r2(arousal_ts1,arousal_pred_ts1)))
print('2. Training r2 score :' + str(r2(arousal_tr2,arousal_pred_tr2)) + ' Test r2 score :' + str(r2(arousal_ts2,arousal_pred_ts2)))
print('3. Training r2 score :' + str(r2(arousal_tr3,arousal_pred_tr3)) + ' Test r2 score :' + str(r2(arousal_ts3,arousal_pred_ts3)))

1. Training r2 score :0.016519718292420826 Test r2 score :-0.011082174771163356
2. Training r2 score :-0.014678259822320738 Test r2 score :0.012829925743721438
3. Training r2 score :0.010317637712854322 Test r2 score :-0.14234550265047452


In [22]:
#Valence Error(mae) vs dataset 1,2,3 
print('1. Training Error :' + str(mae(valence_tr1,valence_pred_tr1)) + ' Test Error :' + str(mae(valence_ts1,valence_pred_ts1)))
print('2. Training Error :' + str(mae(valence_tr2,valence_pred_tr2)) + ' Test Error :' + str(mae(valence_ts2,valence_pred_ts2)))
print('3. Training Error :' + str(mae(valence_tr3,valence_pred_tr3)) + ' Test Error :' + str(mae(valence_ts3,valence_pred_ts3)))

1. Training Error :0.402983002878274 Test Error :0.41855571292566535
2. Training Error :0.41635793959157485 Test Error :0.4357292795836022
3. Training Error :0.3950621861230695 Test Error :0.4083767005891511


In [23]:
#Valence Error(mse) vs dataset 1,2,3 
print('1. Training Error :' + str(mse(valence_tr1,valence_pred_tr1)) + ' Test Error :' + str(mse(valence_ts1,valence_pred_ts1)))
print('2. Training Error :' + str(mse(valence_tr2,valence_pred_tr2)) + ' Test Error :' + str(mse(valence_ts2,valence_pred_ts2)))
print('3. Training Error :' + str(mse(valence_tr3,valence_pred_tr3)) + ' Test Error :' + str(mse(valence_ts3,valence_pred_ts3)))

1. Training Error :0.1826353023605334 Test Error :0.1998948941962561
2. Training Error :0.18987273319341452 Test Error :0.21111774380126966
3. Training Error :0.18168619888081694 Test Error :0.19674231581081406


In [24]:
#Valence r2 score(r2) vs dataset 1,2,3 
print('1. Training r2 score :' + str(r2(valence_tr1,valence_pred_tr1)) + ' Test r2 score :' + str(r2(valence_ts1,valence_pred_ts1)))
print('2. Training r2 score :' + str(r2(valence_tr2,valence_pred_tr2)) + ' Test r2 score :' + str(r2(valence_ts2,valence_pred_ts2)))
print('3. Training r2 score :' + str(r2(valence_tr3,valence_pred_tr3)) + ' Test r2 score :' + str(r2(valence_ts3,valence_pred_ts3)))

1. Training r2 score :0.0012937934414397612 Test r2 score :-0.05335110703436996
2. Training r2 score :-0.0382827122993179 Test r2 score :-0.11249019161706486
3. Training r2 score :0.006483784224207589 Test r2 score :-0.03673851697462749


# Save the data for vizualization and model comparision

In [29]:
#Arousal test data -- for viz
np.savetxt("Vizdata/ANN_arousal1_groundthruth-pred.csv", np.column_stack((arousal_ts1.values,arousal_pred_ts1)), delimiter=',', header="Ground truth value,Predicted value", comments="")
np.savetxt("Vizdata/ANN_arousal2_groundthruth-pred.csv", np.column_stack((arousal_ts1.values,arousal_pred_ts1)), delimiter=',', header="Ground truth value,Predicted value", comments="")
np.savetxt("Vizdata/ANN_arousal3_groundthruth-pred.csv", np.column_stack((arousal_ts1.values,arousal_pred_ts1)), delimiter=',', header="Ground truth value,Predicted value", comments="")
#Arousal test error -- for model comp
arousal_mse = np.column_stack((mse(arousal_ts1,arousal_pred_ts1),mse(arousal_ts2,arousal_pred_ts2),mse(arousal_ts3,arousal_pred_ts3)))
np.savetxt("Compdata/ANN_arousal_performance-mse.csv", arousal_mse, delimiter=',', header="MSE1,MSE2,MSE3", comments="")
arousal_mae = np.column_stack((mae(arousal_ts1,arousal_pred_ts1),mae(arousal_ts2,arousal_pred_ts2),mae(arousal_ts3,arousal_pred_ts3)))
np.savetxt("Compdata/ANN_arousal_performance-mae.csv", arousal_mae, delimiter=',', header="MAE1,MAE2,MAE3", comments="")
arousal_r2 = np.column_stack((r2(arousal_ts1,arousal_pred_ts1),r2(arousal_ts2,arousal_pred_ts2),r2(arousal_ts3,arousal_pred_ts3)))
np.savetxt("Compdata/ANN_arousal_performance-r2.csv", arousal_r2, delimiter=',', header="r2-1,r2-2,r2-3", comments="")

In [30]:
#Valence test data -- for viz
np.savetxt("Vizdata/ANN_valence1_groundthruth-pred.csv", np.column_stack((valence_ts1.values,valence_pred_ts1)), delimiter=',', header="Ground truth value,Predicted value", comments="")
np.savetxt("Vizdata/ANN_valence2_groundthruth-pred.csv", np.column_stack((valence_ts1.values,valence_pred_ts1)), delimiter=',', header="Ground truth value,Predicted value", comments="")
np.savetxt("Vizdata/ANN_valence3_groundthruth-pred.csv", np.column_stack((valence_ts1.values,valence_pred_ts1)), delimiter=',', header="Ground truth value,Predicted value", comments="")
#Valence test error -- for model comp
valence_mse = np.column_stack((mse(valence_ts1,valence_pred_ts1),mse(valence_ts2,valence_pred_ts2),mse(valence_ts3,valence_pred_ts3)))
np.savetxt("Compdata/ANN_valence_performance-mse.csv", valence_mse, delimiter=',', header="MSE1,MSE2,MSE3", comments="")
valence_mae = np.column_stack((mae(valence_ts1,valence_pred_ts1),mae(valence_ts2,valence_pred_ts2),mae(valence_ts3,valence_pred_ts3)))
np.savetxt("Compdata/ANN_valence_performance-mae.csv", valence_mae, delimiter=',', header="MAE1,MAE2,MAE3", comments="")
valence_r2 = np.column_stack((r2(valence_ts1,valence_pred_ts1),r2(valence_ts2,valence_pred_ts2),r2(valence_ts3,valence_pred_ts3)))
np.savetxt("Compdata/ANN_valence_performance-r2.csv", valence_r2, delimiter=',', header="r2-1,r2-2,r2-3", comments="")